UUDBのパラ言語情報（6種類の実数値）を予測するモデルを作成するためのデータセットを作成する．

In [1]:
from datasets import Dataset, Audio, DatasetDict
import pandas as pd
import pathlib
import os
from typing import List

最初の方は過去の方法と同様

In [2]:
# UUDBコーパスのあるディレクトリ
UUDB_TOP_DIR="/autofs/diamond2/share/corpus/UUDB/UUDB"

In [3]:
# 話者情報の読み込み
speaker_info_df = pd.read_csv('uudb_speaker_info.csv', index_col=0)
# speaker_info_df['L']['C001'] で C001 の L　の話者のspeaker_id (FTSなど) が得られる

In [4]:
df_list = []
for dir in sorted(pathlib.Path(UUDB_TOP_DIR + "/var").glob('C*')):
    session_name = os.path.basename(dir)
    # list と para のファイルパス
    list_file_path = os.path.join(dir.as_posix(), session_name + ".list")
    para_file_path = os.path.join(dir.as_posix(), session_name + ".para")
    # list と para を DataFrame に読み込む
    list_df = pd.read_table(list_file_path, header=None)
    list_df.columns = ["wav"]
    para_df = pd.read_table(para_file_path, header=None)
    para_df.columns = ["pleasantness", "arousal", "dominance", "credibility", "interest", "positivity"]
    # wavファイル名から必要な情報を抽出して列にする. session, L or R, 発話の通し番号
    list_session = list_df['wav'].apply(lambda x: x[:4])
    list_session.name = 'session'
    list_channel = list_df['wav'].apply(lambda x: x[4])
    list_channel.name = 'channel'
    list_utterance_number = list_df['wav'].apply(lambda x: int(x[6:9]))
    list_utterance_number.name = 'utterance_number'
    # speaker_id
    speaker_id = dict(L=speaker_info_df['L'][session_name], R=speaker_info_df['R'][session_name])
    list_speaker_id = list_df['wav'].apply(lambda x: speaker_id[x[4]])
    list_speaker_id.name = "speaker"
    # 以上を結合する
    df = pd.concat([list_df, list_session, list_channel, 
                    list_utterance_number, list_speaker_id, para_df], axis=1)
    df_list.append(df)
# 全体を一つに結合
info_df = pd.concat(df_list, axis=0, ignore_index=True)

In [5]:
# 全セッション名のリスト
session_list = speaker_info_df.index.tolist()
# 学習, 検証, テストのいずれにも使わないデータのセッション名のリスト
ignore_session_list = ["C061", "C062", "C063", "C064"]
# 検証データのセッション名のリスト
validation_session_list = ["C051"]
# テストデータのセッション名のリスト
test_session_list = ["C052", "C053"]
# 学習データのセッション名のリスト
train_session_list = [session for session in session_list if session not in ignore_session_list + validation_session_list + test_session_list]

音声を含むデータセットを作成指定する関数

In [6]:
def make_dataset_from_df(df: pd.DataFrame, session_list: List[str]) -> Dataset:
    sub_df = info_df[info_df['session'].isin(session_list)]
    df_dict = sub_df.to_dict(orient='list')
    # train_df_dict の wav キーを audio キーに変更
    df_dict['audio'] = df_dict.pop('wav')
    # audio の項のファイルパスをフルパスに変更
    # foo.wav を UUDB_TOP_DIR/var/C001/C001_foo.wav に変更
    df_dict['audio'] = [os.path.join(UUDB_TOP_DIR, 'var', session, wav) for session, wav in zip(df_dict['session'], df_dict['audio'])]
    # dataset化
    dataset = Dataset.from_dict(df_dict).cast_column('audio', Audio())

    return dataset

上記の関数で各学習データを作成

In [7]:
train_dataset = make_dataset_from_df(info_df, train_session_list)
validation_dataset = make_dataset_from_df(info_df, validation_session_list)
test_dataset = make_dataset_from_df(info_df, test_session_list)

ディスクに保存する

In [8]:
# Diskに保存
DatasetDict(train=train_dataset, validation=validation_dataset, test=test_dataset).save_to_disk('uudb_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/3334 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/203 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/169 [00:00<?, ? examples/s]